In [24]:
import kagglehub
import json
import pandas as pd
import numpy as np
import cv2
import os

# Dataset Filtering

#### 1. Downloading the Latest version of the Dataset

In [25]:

# Download latest version
path = kagglehub.dataset_download("risangbaskoro/wlasl-processed")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Sahil\.cache\kagglehub\datasets\risangbaskoro\wlasl-processed\versions\5


##

In [26]:
# Importing the dataset

with open('C:/Users/Sahil/.cache/kagglehub/datasets/risangbaskoro/wlasl-processed/versions/5/WLASL_v0.3.json' , 'r') as file:

 data = json.load(file)

In [27]:
data[1999]['instances'][0]['bbox']

[0, 0, 360, 240]

In [28]:
#Flattening the data so that each instance can be represented by a row in DataFrame

flat_data = [] 

for i in range(0 , len(data)):

 for instance in data[i]['instances']:
   
   instance_data = {
    "gloss" : data[i]['gloss'],
    "bbox": instance["bbox"],
    "fps": instance["fps"],
    "frame_end": instance["frame_end"],
    "frame_start": instance["frame_start"],
    "instance_id": instance["instance_id"],
    "signer_id": instance["signer_id"],
    "source": instance["source"],
    "split": instance["split"],
    "url": instance["url"],
    "variation_id": instance["variation_id"],
    "video_id": instance["video_id"]
   }

   flat_data.append(instance_data)

In [29]:
sign_data_df = pd.DataFrame(flat_data)

In [30]:
sign_data_df

,gloss,bbox,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,video_id
0,book,"[385, 37, 885, 720]",25,-1,1,0,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/book.mp4,0,69241
1,book,"[190, 25, 489, 370]",25,-1,1,1,90,aslsignbank,train,https://aslsignbank.haskins.yale.edu/dictionar...,0,65225
2,book,"[262, 1, 652, 480]",25,-1,1,2,110,valencia-asl,train,https://www.youtube.com/watch?v=0UsjUE-TXns,0,68011
3,book,"[123, 19, 516, 358]",25,60,1,3,113,lillybauer,train,https://www.youtube.com/watch?v=1QOYOZ3g-aY,0,68208
4,book,"[95, 0, 1180, 720]",25,-1,1,4,109,valencia-asl,train,https://www.youtube.com/watch?v=aGtIHKEdCds,0,68012
...,...,...,...,...,...,...,...,...,...,...,...,...
21078,whistle,"[76, 17, 236, 240]",25,-1,1,2,2,spreadthesign,train,https://media.spreadthesign.com/video/mp4/13/9...,0,63186
21079,whistle,"[3, 0, 320, 240]",25,-1,1,3,14,handspeak,train,https://www.handspeak.com/word/w/whistle.mp4,0,63187
21080,whistle,"[68, 14, 212, 192]",25,-1,1,4,11,signingsavvy,train,https://www.signingsavvy.com/signs/mp4/9/9961.mp4,0,63188
21081,whistle,"[18, 4, 228, 240]",25,-1,1,5,84,aslpro,train,http://www.aslpro.com/main/w/whistle.swf,0,63189


In [31]:
type(sign_data_df['video_id'][0])

str

#### 2. Removing videos from dataset not present in the Local Folders

In [34]:
#Function to remove the records whose videos are not present

def check_video_availability(vid_id , video_directory):

 video_path = os.path.join(video_directory ,  vid_id + ".mp4")

 return os.path.exists(video_path)


In [35]:
# Defining the directory where the video is stored

video_directory = r'C:\Users\Sahil\Desktop\versions\5\videos'

# Applying the function to all the records in the dataframe and creating a new attrbute for the records

sign_data_df['is_available'] = sign_data_df['video_id'].apply(lambda vid : check_video_availability(vid , video_directory))

In [36]:
sign_data_df

,gloss,bbox,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,video_id,is_available
0,book,"[385, 37, 885, 720]",25,-1,1,0,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/book.mp4,0,69241,True
1,book,"[190, 25, 489, 370]",25,-1,1,1,90,aslsignbank,train,https://aslsignbank.haskins.yale.edu/dictionar...,0,65225,False
2,book,"[262, 1, 652, 480]",25,-1,1,2,110,valencia-asl,train,https://www.youtube.com/watch?v=0UsjUE-TXns,0,68011,False
3,book,"[123, 19, 516, 358]",25,60,1,3,113,lillybauer,train,https://www.youtube.com/watch?v=1QOYOZ3g-aY,0,68208,False
4,book,"[95, 0, 1180, 720]",25,-1,1,4,109,valencia-asl,train,https://www.youtube.com/watch?v=aGtIHKEdCds,0,68012,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21078,whistle,"[76, 17, 236, 240]",25,-1,1,2,2,spreadthesign,train,https://media.spreadthesign.com/video/mp4/13/9...,0,63186,True
21079,whistle,"[3, 0, 320, 240]",25,-1,1,3,14,handspeak,train,https://www.handspeak.com/word/w/whistle.mp4,0,63187,False
21080,whistle,"[68, 14, 212, 192]",25,-1,1,4,11,signingsavvy,train,https://www.signingsavvy.com/signs/mp4/9/9961.mp4,0,63188,True
21081,whistle,"[18, 4, 228, 240]",25,-1,1,5,84,aslpro,train,http://www.aslpro.com/main/w/whistle.swf,0,63189,False


In [44]:
sign_final_df = sign_data_df[sign_data_df['is_available'] == True]

In [48]:
len(sign_final_df)

11980

#### 3.Storing as a CSV file to store it in MongoDB

In [49]:
sign_final_df.to_csv('Final_Sign_Language_Data.csv' , index= False)

In [23]:
def get_total_frames(video_path):

 #open the video 

 cap = cv2.VideoCapture(video_path)

 if not cap.isOpened():
  print('not opened')
  return 0
 
 # Get the total number of frames in the video
 total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
 # Release the video capture object
 cap.release()


    
 return total_frames